In [6]:
from bs4 import BeautifulSoup
import re

In [7]:
DATA_HOME = "foodbase/"
data = open("%sFoodBase_curated.xml" % DATA_HOME).read()

In [8]:
soup = BeautifulSoup(data, 'lxml')

In [9]:
annotations = soup.find_all("annotation")

In [10]:
text = annotations[0].find('text').text

In [11]:
classes = annotations[0].find('infon').text.split(';')
classes.remove('')

In [12]:
pattern = re.compile(r"\s*(\S+)\s+\[(.+)\]")

In [13]:
pattern.findall(classes[0])

[('AG.01.e', 'Dairy produce')]

In [14]:
all_classes = {
}

processed_annotations = []

for annotation in annotations:
    text = annotation.find('text').text
    classes = annotation.find('infon').text.split(';')
    classes.remove('')
    cleaned_classes = []
    for classs in classes:
        groups = pattern.findall(classs)
        if len(groups) > 0:
            groups = groups[0]
        else:
            continue
        all_classes[groups[0]] = groups[1]
        cleaned_classes.append({
            "id": groups[0],
            "name": groups[1]
        })
    processed_annotations.append({
        "text": text,
        "class": cleaned_classes
    })
    

In [15]:
all_classes

{'AG.01.e': 'Dairy produce',
 'AG.01.e.02': 'Cheese',
 'AG.01.n': 'Dishes and prepared food',
 'AG.01.n.18': 'Preserve',
 'AG.01.d.03': 'Beef',
 'AG.01.h.01.e': 'Fruit containing stone',
 'AG.01.h.02.e': 'Onion/leek/garlic',
 'AG.01.h': 'Fruit and vegetables',
 'AG.01.l.04': 'Sauce/dressing',
 'AG.01.h.01.f': 'Nut',
 'AG.01.h.02.b': 'Stalk vegetables',
 'AG.01.n.01': 'Food by way of preparation',
 'AG.01.n.11': 'Bread',
 'AG.01.n.12': 'Pancake/tortilla/oatcake',
 'AG.01': 'Food',
 'AG.01.z': 'Water',
 'AG.01.h.02.f': 'Fruits as vegetables',
 'AG.01.af': 'Tea manufacture',
 'AG.01.f': 'Fat/oil',
 'AG.01.l.01': 'Salt',
 'AG.01.w': 'Setting table',
 'AG.01.h.02.g': 'Pulse',
 'AG.01.ab': 'Fruit juice/squash',
 'AG.01.h.01.a': 'Citrus fruit',
 'AG.01.l.02': 'Sweetener (syrup/honey/chocolate)',
 'AF.20.f': 'Pulses/plants producing pulses',
 'AG.01.d.06': 'Fowls',
 'AG.01.i': 'Corn/cereals/grain',
 'AG.01.d': 'Animals for food',
 'AG.01.t.08': 'Equipment for food preparation',
 'AG.01.t.06': 

In [19]:
import pickle

pickle.dump(all_classes, open('classes.dump', 'wb'))
pickle.dump(processed_annotations, open('annot.dump', 'wb'))

In [16]:
import pandas as pd

In [65]:
annot = pd.DataFrame(processed_annotations)

In [66]:
annot = annot.drop_duplicates('text')
annot['text'] = annot['text'].apply(lambda x: x.lower())
annot = annot.sort_values('text')

In [67]:
annot.to_csv('annotations.csv')

In [2]:
df = pd.read_csv('annotations.csv')